# Generating pitch tracks for the Guše visualizer

In [12]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [13]:
folderName = './'
mbid = '37acb317-3ef0-4f91-ad1f-36d793ba7378'
pitchTrackExtension = '.f0_fil.csv'
fileName = mbid + pitchTrackExtension

dur = 76

shahed = 160.83

filePath = os.path.join(folderName, fileName)
print(filePath)

./37acb317-3ef0-4f91-ad1f-36d793ba7378.f0_fil.csv


### From Hz to cents

In [14]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [15]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

array([[  0.55 , 127.411],
       [  0.56 , 128.35 ],
       [  0.59 , 148.707],
       ...,
       [ 74.93 , 165.131],
       [ 74.94 , 164.565],
       [ 74.95 , 163.926]])

In [16]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

Max: 4163.0
Min: -1207.0


In [17]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [18]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

dur

76

In [19]:
maxCent = 1000
minCent = -400
minCut = True

In [20]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        c = h2c(shahed, pitchTrackDic[k])
        if minCut:
            if c >= minCent and c <= maxCent:
                pitchTrackCents[k] = c
        else:
            pitchTrackCents[k] = c

pitchTrackCents

{'0.00': 's',
 '0.01': 's',
 '0.02': 's',
 '0.03': 's',
 '0.04': 's',
 '0.05': 's',
 '0.06': 's',
 '0.07': 's',
 '0.08': 's',
 '0.09': 's',
 '0.10': 's',
 '0.11': 's',
 '0.12': 's',
 '0.13': 's',
 '0.14': 's',
 '0.15': 's',
 '0.16': 's',
 '0.17': 's',
 '0.18': 's',
 '0.19': 's',
 '0.20': 's',
 '0.21': 's',
 '0.22': 's',
 '0.23': 's',
 '0.24': 's',
 '0.25': 's',
 '0.26': 's',
 '0.27': 's',
 '0.28': 's',
 '0.29': 's',
 '0.30': 's',
 '0.31': 's',
 '0.32': 's',
 '0.33': 's',
 '0.34': 's',
 '0.35': 's',
 '0.36': 's',
 '0.37': 's',
 '0.38': 's',
 '0.39': 's',
 '0.40': 's',
 '0.41': 's',
 '0.42': 's',
 '0.43': 's',
 '0.44': 's',
 '0.45': 's',
 '0.46': 's',
 '0.47': 's',
 '0.48': 's',
 '0.49': 's',
 '0.50': 's',
 '0.51': 's',
 '0.52': 's',
 '0.53': 's',
 '0.54': 's',
 '0.56': -391.0,
 '0.57': 's',
 '0.58': 's',
 '0.59': -136.0,
 '0.60': -120.0,
 '0.61': -95.0,
 '0.62': -60.0,
 '0.63': 's',
 '0.64': 's',
 '0.65': 168.0,
 '0.66': 217.0,
 '0.67': 242.0,
 '0.68': 249.0,
 '0.69': 245.0,
 '0.70': 23

In [21]:
with open(mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

In [22]:
x = 0
y = 0
for k in pitchTrackCents.keys():
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] > x:
        x = pitchTrackCents[k]
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] < y:
        y = pitchTrackCents[k]
print(y, x)

-391.0 936.0


### Pitch histogram

In [ ]:
import intonation
import numpy as np
import json
import os

In [ ]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [ ]:
# folderName = './'
# mbid = '7d88c651-5dea-48e5-b21a-ac964cff5cad'
# pitchTrackExtension = '.f0_fil.csv'

# shahed = 332.67

In [ ]:
pitchTrack = np.genfromtxt(os.path.join(folderName, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
filteredTrack = centsTrack[np.logical_and(centsTrack >= minCent, centsTrack <= maxCent)]
pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
# pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

In [ ]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

In [ ]:
pitchHistogramFolder = './'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)